In [1]:
import pandas as pd
import re
import numpy as np
from scipy.stats import ttest_ind

In [2]:
NY_crime_data = pd.read_csv("Hate_Crimes_by_County_and_Bias_Type__Beginning_2010.csv")
NY_crime_data.head(30)

,County,Year,Crime Type,Anti-Male,Anti-Female,Anti-Transgender,Anti-Gender Identity Expression,Anti-Age*,Anti-White,Anti-Black,...,Anti-Gay Male,Anti-Gay Female,Anti-Gay (Male and Female),Anti-Heterosexual,Anti-Bisexual,Anti-Physical Disability,Anti-Mental Disability,Total Incidents,Total Victims,Total Offenders
0,Albany,2010,Crimes Against Persons,0,0,0,0,0,0,9,...,2,1,0,0,0,0,0,13,13,20
1,Albany,2010,Property Crimes,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2,2,2
2,Albany,2011,Crimes Against Persons,0,0,0,0,0,0,4,...,2,0,0,0,0,0,0,7,7,7
3,Albany,2011,Property Crimes,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,3,3,3
4,Albany,2012,Crimes Against Persons,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,5,5,8
5,Albany,2012,Property Crimes,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,3,3,3
6,Albany,2013,Crimes Against Persons,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,3,3,3
7,Albany,2013,Property Crimes,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2,2,2
8,Albany,2014,Crimes Against Persons,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,3,3,3
9,Albany,2014,Property Crimes,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,1


In [3]:
FIPS = pd.read_csv("New_York_State_ZIP_Codes-County_FIPS_Cross-Reference.csv")
FIPS.head(30)

,County Name,State FIPS,County Code,County FIPS,ZIP Code,File Date
0,Albany,36,1,36001,12007,07/25/2007
1,Albany,36,1,36001,12009,07/25/2007
2,Albany,36,1,36001,12023,07/25/2007
3,Albany,36,1,36001,12041,07/25/2007
4,Albany,36,1,36001,12045,07/25/2007
5,Albany,36,1,36001,12046,07/25/2007
6,Albany,36,1,36001,12047,07/25/2007
7,Albany,36,1,36001,12053,07/25/2007
8,Albany,36,1,36001,12054,07/25/2007
9,Albany,36,1,36001,12055,07/25/2007


In [4]:
FIPS1 = FIPS.drop_duplicates(subset =["County Name"], 
                     keep = 'first') 

In [5]:
FIPS2 = FIPS1.drop(columns=['ZIP Code'])

In [6]:
FIPS2

,County Name,State FIPS,County Code,County FIPS,File Date
0,Albany,36,1,36001,07/25/2007
86,Allegany,36,3,36003,07/25/2007
125,Bronx,36,5,36005,07/25/2007
155,Broome,36,7,36007,07/25/2007
193,Cattaraugus,36,9,36009,07/25/2007
233,Cayuga,36,11,36011,07/25/2007
262,Chautauqua,36,13,36013,07/25/2007
302,Chemung,36,15,36015,07/25/2007
325,Chenango,36,17,36017,07/25/2007
355,Clinton,36,19,36019,07/25/2007


In [7]:
hate_crime = NY_crime_data.merge(FIPS2, left_on='County', right_on='County Name')
hate_crime

,County,Year,Crime Type,Anti-Male,Anti-Female,Anti-Transgender,Anti-Gender Identity Expression,Anti-Age*,Anti-White,Anti-Black,...,Anti-Physical Disability,Anti-Mental Disability,Total Incidents,Total Victims,Total Offenders,County Name,State FIPS,County Code,County FIPS,File Date
0,Albany,2010,Crimes Against Persons,0,0,0,0,0,0,9,...,0,0,13,13,20,Albany,36,1,36001,07/25/2007
1,Albany,2010,Property Crimes,0,0,0,0,0,0,1,...,0,0,2,2,2,Albany,36,1,36001,07/25/2007
2,Albany,2011,Crimes Against Persons,0,0,0,0,0,0,4,...,0,0,7,7,7,Albany,36,1,36001,07/25/2007
3,Albany,2011,Property Crimes,0,0,0,0,0,0,1,...,0,0,3,3,3,Albany,36,1,36001,07/25/2007
4,Albany,2012,Crimes Against Persons,0,0,0,0,0,0,1,...,0,0,5,5,8,Albany,36,1,36001,07/25/2007
5,Albany,2012,Property Crimes,0,0,0,0,0,0,1,...,0,0,3,3,3,Albany,36,1,36001,07/25/2007
6,Albany,2013,Crimes Against Persons,0,0,0,0,0,1,0,...,0,0,3,3,3,Albany,36,1,36001,07/25/2007
7,Albany,2013,Property Crimes,0,0,0,0,0,0,1,...,0,0,2,2,2,Albany,36,1,36001,07/25/2007
8,Albany,2014,Crimes Against Persons,0,0,0,0,0,0,1,...,0,0,3,3,3,Albany,36,1,36001,07/25/2007
9,Albany,2014,Property Crimes,0,0,0,0,1,0,0,...,0,0,1,1,1,Albany,36,1,36001,07/25/2007


In [8]:
hate_crime1 = hate_crime.drop(columns=['File Date'])
hate_crime1.loc[[118]]

,County,Year,Crime Type,Anti-Male,Anti-Female,Anti-Transgender,Anti-Gender Identity Expression,Anti-Age*,Anti-White,Anti-Black,...,Anti-Bisexual,Anti-Physical Disability,Anti-Mental Disability,Total Incidents,Total Victims,Total Offenders,County Name,State FIPS,County Code,County FIPS
118,Genesee,2013,Crimes Against Persons,0,0,0,0,0,0,1,...,0,0,0,1,1,1,Genesee,36,37,36037


In [9]:
hate_crime1.dtypes

County                                          object
Year                                             int64
Crime Type                                      object
Anti-Male                                        int64
Anti-Female                                      int64
Anti-Transgender                                 int64
Anti-Gender Identity Expression                  int64
Anti-Age*                                        int64
Anti-White                                       int64
Anti-Black                                       int64
Anti-American Indian/Alaskan Native              int64
Anti-Asian                                       int64
Anti-Native Hawaiian/Pacific Islander            int64
Anti-Multi-Racial Groups                         int64
Anti-Other Race                                  int64
Anti-Jewish                                      int64
Anti-Catholic                                    int64
Anti-Protestant                                  int64
Anti-Islam

In [12]:
hate_crime1['County FIPS'] = hate_crime1['County FIPS'].astype(str)

In [13]:
hate_crime1['County FIPS'] = hate_crime1['County FIPS'].str.replace('^\d\d', '', regex=True)

In [ ]:
#hate_crime1['County FIPS'] = hate_crime1['County FIPS'].astype(int)

In [20]:
pd.DataFrame(hate_crime1.dtypes).reset_index()[0].to_list()

[dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('O')]

In [ ]:
#hate_crime1.to_csv('ny-hate-crime.csv')

In [23]:
hate_crime1.sort_values(by=['Total Offenders'], ascending=False).head(10)

,County,Year,Crime Type,Anti-Male,Anti-Female,Anti-Transgender,Anti-Gender Identity Expression,Anti-Age*,Anti-White,Anti-Black,...,Anti-Bisexual,Anti-Physical Disability,Anti-Mental Disability,Total Incidents,Total Victims,Total Offenders,County Name,State FIPS,County Code,County FIPS
192,New York,2013,Crimes Against Persons,0,0,0,1,0,4,6,...,0,1,0,72,80,113,New York,36,61,061
133,Kings,2012,Property Crimes,0,0,0,0,0,1,4,...,0,0,0,101,106,104,Kings,36,47,047
138,Kings,2015,Crimes Against Persons,0,0,0,0,0,3,6,...,0,0,0,78,81,104,Kings,36,47,047
134,Kings,2013,Crimes Against Persons,0,0,0,0,0,4,4,...,0,0,0,68,76,102,Kings,36,47,047
128,Kings,2010,Crimes Against Persons,0,0,0,0,0,3,10,...,0,0,0,81,82,101,Kings,36,47,047
136,Kings,2014,Crimes Against Persons,0,0,0,0,1,1,4,...,0,0,0,72,76,97,Kings,36,47,047
186,New York,2010,Crimes Against Persons,1,0,0,0,0,1,5,...,0,0,0,75,75,91,New York,36,61,061
132,Kings,2012,Crimes Against Persons,0,0,0,0,0,1,6,...,0,0,0,56,62,87,Kings,36,47,047
196,New York,2015,Crimes Against Persons,0,0,0,0,0,3,6,...,0,0,0,67,73,86,New York,36,61,061
353,Suffolk,2014,Property Crimes,0,0,0,0,9,0,13,...,0,0,0,75,75,84,Suffolk,36,103,103
